### 拓扑排序
&emsp;&emsp;在学习拓扑排序之前，首先将图的bfs和dfs复习一下

#### 1.广度优先搜索 
&emsp;&emsp;该算法始终是将已发现节点和未发现节点之间的边界，沿其广度方向向外扩展，也就是说，算法需要在发现所有距离源节点s为k的所有节点之后，才会发现距离源节点s为k+1的其他节点。

&emsp;&emsp;为了跟踪算法的进展，广度优先搜索在概念上将每个节点涂上白色,灰色,黑色，所有节点在一开始的时候涂上白色，在算法推进的过程中，这些节点可能会变成灰色或者黑色，在搜索的过程中，第一次遇到一个节点就称为该节点被发现，此时该节点的颜色将发生变化，因此凡是黑色或者灰色的节点都是已被发现的节点。但广度优先搜索对灰色和黑色节点加以区分，以确保搜索按照广度优先模式进行推进，如果边(u,v)属于E，且节点u是黑色的，则节点v既可能是灰色的也可能是黑色的，也就是说，所有与黑色节点临接的节点都已经被发现，对于灰色节点来说，其临接节点中可能存在未被发现的白色节点。灰色节点所代表的就是已知和未知两个集合之间的边界。

&emsp;&emsp;在执行广度优先搜索的过程中将构造一颗广度优先树，一开始，该树仅含有根节点，就是源节点s，在扫描以发现节点u的邻接链表时，每当发现一个白色节点v，就将节点v和边(u,v)同时加入该棵树中，在广度优先树中，称节点u是节点v的前驱或者父节点，由于每个节点最多被发现一次，它最多只有一个父节点，广度优先树中的祖先和后代关系皆以相对于根节点s的位置来进行定义：如果节点u是从根节点s到节点v的简单路径上的一个节点，则节点u是节点v的祖先，节点v是节点u的后代。

&emsp;&emsp;在下面给出的广度优先搜索过程中，假定输入图G = (V,E)是以邻接表所表示的，该算法为图中每个节点赋予一些额外的属性:每个节点u的颜色放在属性u.color里，将u的前驱节点放在属性u.prev里，如果u没有前驱节点，则u.prev = None，属性u.distance记录的是广度优先搜索算法所计算出的从源节点s到节点u之间的距离。该算法使用一个先进先出的队列Q来管理灰色节点集合。
伪代码如下：
```
def bfs(G,s):
    for each vertex u in G.V - s:
        u.color = white
        u.distance = float('Inf')
        u.prev = None
    s.color = gray
    s.distance = 0
    s.prev = None
    Q = queue()
    enqueue(Q,s)
    while Q:
        u = dequeue(Q)
        for each v in G.adj[u]:
            if v.color == white:
                v.color = gray
                v.d = u.d + 1
                v.prev = u
                enqueue(Q,v)
        u.color = black
```

In [12]:
class Graph:
    def __init__(self):
        self.vertex_list = []
        
class Vertex:
    def __init__(self,key):
        self.key = key
        self.color = 'white'
        self.d = float('Inf')
        self.prev = None
        self.adj = []
        
def bfs(G,s):
    # 初始化
    for u in G.vertex_list:
        u.color = 'white'
        u.d = float('Inf')
        u.prev = None
        
    # 处理s节点
    s.color = 'gray'
    s.d = 0
    s.prev = None
    
    queue = []
    queue.append(s)
    bfs_list = []
    while queue:
        u = queue.pop(0)
        for v in u.adj:
            if v.color == 'white':
                v.color = 'gray'
                v.d = u.d + 1
                v.prev = u
                queue.append(v)
        u.color = 'black'
        bfs_list.append(u)
    return bfs_list

In [13]:
s = Vertex('s') 
r = Vertex('r') 
v = Vertex('v') 
w = Vertex('w') 
t = Vertex('t')
x = Vertex('x') 
u = Vertex('u')
y = Vertex('y') 


s.adj = [r, w]
r.adj = [s, v]
v.adj = [r]
w.adj = [s, t,x]
t.adj = [x, w,u]
x.adj = [t, w,u,y]
u.adj = [t, x,y]
y.adj = [x, u]


graph = Graph()
graph.vertex_list = [s,r,w,v,t,x,u,y]

bfs_list = bfs(graph,s)

In [14]:
for node in bfs_list:
    print(node.key + ' | ' + str(node.d))

s | 0
r | 1
w | 1
v | 2
t | 2
x | 2
u | 3
y | 3


&emsp;&emsp;下图描述的是BFS在一个样本图上的推进过程。

![bfs图解](./pic/topological_sort/bfs.jpg)

&emsp;&emsp;**广度优先树**

&emsp;&emsp;bfs在对图进行搜索的过程中将创建一颗广度优先树，如上图所示，该棵树对应的是prev属性；对于图G = (V,E)和源节点s，定义图的前驱子图为$G_{prev} = (V_{prev},E_{prev})$,其中$V_{prev} = {v \in V:v.prev != None} \cup {s},E_{prev} = {(v.prev,v):v \in V_{prev} - {s} }$

&emsp;&emsp;如果$V_{prev}$由从源节点s可以到达的节点组成，并且对于所有的$v \in V_{prev}$，子图$G_{prev}$包含一条从源节点s到节点v的唯一简单路径，且该路径也是图G里面从源节点s到节点v之间的一条最短路径，则前驱子图$G_{prev}$是一颗广度优先树。广度优先树实际上就是一颗树，因为它是连通的，并且$|E_{prev}| = |V_{prev}| - 1$，称$E_{prev}$中的边为树边。

#### 2.深度优先搜索 
&emsp;&emsp;深度优先搜索总是对最近才发现的节点v的出发边进行探索，直到该节点的所有出发边都被发现为止。一旦节点v的所有出发边都被发现，搜索则回溯到v的前驱节点，来搜索该前驱节点的出发边。该过程一直持续到从愿节点可以达到的所有节点都被发现为止。如果还存在尚未发现的节点,则深度优先搜索将从这些未被发现的节点中任选一个作为新的源节点,并重复同样的搜索过程，该算法重复整个过程，直到图中的所有节点都被发现为止。

&emsp;&emsp;像广度优先搜索一样，在对已被发现的节点u的邻接链表进行扫描时，每当发现一个节点v时，深度优先搜索算法将对这个时间进行记录，将v的前驱属性v.prev 设置为u,不过与广度优先搜索不同的是，广度优先搜索的前驱子图是一颗树，而深度优先搜索的前驱子图可能由多颗树组成，因为搜索可能从多个源节点重复进行。因此，给深度优先搜索的前驱子图所下的定义与对广度优先搜索前驱子图所下的定义略有不同；设图$G_{prev} = (V,E_{prev})$ ,其中$E_{prev} = {(v.prev,v):v \in V \cup v.prev != None}$，深度优先搜索的前驱子图形成一个由多棵深度优先树构成的深度优先森林，森林$E_{prev}$中的边仍然成为树边。

&emsp;&emsp;与广度优先搜索相似，深度优先搜索算法在搜索的过程中也对节点进行涂色来指明节点的状态。每个节点的初始颜色都是白色，在节点被发现后变为灰色，在其邻接表被扫描完成后变为黑色，该方法可以保证每个节点仅在一颗深度优先树中出现，因此所有的深度优先树是不相交的。

&emsp;&emsp;除了创建一个深度优先搜索森林外，深度优先搜索算法还在每个节点上盖上一个时间戳，每个节点v由两个时间戳，第一个时间戳v.d记录节点v第一次被发现第时间（涂上灰色的时间），第二个时间错v.f记录的是搜索完成对v的邻接链表扫描的时间（涂上黑色的时间），这些时间戳提供了涂结构的重要信息，通常能够帮助推断深度优先搜索算法的行为。

&emsp;&emsp;下面的深度优先搜索算法的伪代码将其发现节点u的时刻记录在属性u.d中，将其完成对节点u处理的时刻记录在属性u.f中，因为|V|个节点中每个节点只能有一个发现时间和完成时间，所以这些时间戳都是处在 1 和 2|V|之间的整数，很显然，对于每个节点u，有u.d < u.f 。节点u在节点u.d之前为白色，在时刻u.d和u.f之间为灰色，在时刻u.f之后为黑色。

&emsp;&emsp;下面的伪代码给出的是基本的深度优先搜索算法。输入图G既可是有向图，也可以是无向图，变量time是一个全局变量，用来计算时间戳。
```
global time = 0
def dfs(G):
    for each vertex v in G.V:
        u.color = white
        u.prev = None

for each vertex u in G.V:
    if u.color == white:
        dfs_visit(G,u)
        
def dfs_visit(G,u):
    time += 1
    u.d = time
    u.color = gray
    for each v in G.adj[u]:
        if v.color == white:
            v.prev = u
            dfs_visit(G,v)
    u.color = black
    time += 1
    u.f = time
```

In [27]:
class Graph:
    def __init__(self):
        self.vertex_list = []

class Vertex:
    def __init__(self,key):
        self.key = key
        self.d = float('Inf')
        self.f = float('Inf')
        self.adj = []
        self.color = 'white'
        self.prev = None
        
class Solution:
    def __init__(self):
        self.time = 0
        self.res = []
    def dfs(self,G):
        for u in G.vertex_list:
            u.color = 'white'
            u.prev = None
        
        for u in G.vertex_list:
            if u.color == 'white':
                self.dfs_visit(G,u)
        return self.res
                
    def dfs_visit(self,G,u):
        self.time += 1
        u.color = 'gray'
        u.d = self.time
        for v in u.adj:
            if v.color == 'white':
                v.prev = u
                self.dfs_visit(G,v)
        u.color = 'black'
        self.time += 1
        u.f = self.time
        self.res.append(u)

In [28]:
u = Vertex('u') 
v = Vertex('v') 

w = Vertex('w') 
x = Vertex('x') 
y = Vertex('y')
z = Vertex('z') 



u.adj = [x, v]
v.adj = [y]
w.adj = [y,z]
x.adj = [v]
y.adj = [x]

z.adj = [z]


graph = Graph()
graph.vertex_list = [u,v,w,x,y,z]

s = Solution()
dfs_list = s.dfs(graph)

for node in dfs_list:
    print(node.key + ' start_time:' + str(node.d) + ' ;end_time:' + str(node.f))

y start_time:4 ;end_time:5
v start_time:3 ;end_time:6
x start_time:2 ;end_time:7
u start_time:1 ;end_time:8
z start_time:10 ;end_time:11
w start_time:9 ;end_time:12


&emsp;&emsp;下图描述的是dfs在一个样本图上的推进过程。

![dfs图解](./pic/topological_sort/dfs.jpg)

&emsp;&emsp;**深度优先搜索的性质**

&emsp;&emsp;定理(括号化定理) 在对有向或无向图G = (V,E)进行的任意深度优先搜索中，对于任意两个节点u和节点v来说，下面三种情况只有一种成立：
 * 1.区间[u.d,u.f]和区间[v.d,v.f]完全分离，在深度优先森林中，节点u既不是节点v的后代，节点v也不是节点u的后代。
 * 2.区间[u.d,u.f]完全包含在区间[v.d,v.f]，在深度优先森林中，节点u是节点v的后代。
 * 3.区间[v.d,v.f]完全包含在区间[u.d,u.f]，在深度优先森林中，节点v是节点u的后代。

#### 3.拓扑排序
&emsp;&emsp;本节主要讲述如何使用深度优先搜索来对有向无环图进行拓扑排序，对于一个有向无环图G = (V,E)来说，其拓扑排序是G中所有节点的一种线性次序，该次序满足如下条件：如果图G包含边(u,v)，则节点u在拓扑排序中处于节点v的前面（如果图G包含环路，则不可能排出一个线性次序）。拓扑排序的如下图例子显示：

![topo图解](./pic/topological_sort/topo.jpg)


&emsp;&emsp;算法思想:用DFS遍历整个图，得出个节点的完成时间，然后按完成时间倒序排列就得到了图的拓扑排序序列。


```
def topological_sort(G):
    call dfs(G) to compute finishing times v.f for each vertex v
    as each vertex is finished ,insert it onto the front of a linked list
    return the linked list of vertices
```


&emsp;&emsp;**引理:一个有向图G = (V,E)是无环的当且仅当对其进行的深度优先搜索不产生后向边**
&emsp;&emsp;**拓扑排序算法topological_sort算法生成的是有向无环图的拓扑排序**

&emsp;&emsp;证明 假定在有向无环图G = (V，E）上运行dfs来计算节点的完成时间。只需证明，对于任意一对不同的节点$u,v \in V$，如果图G包含一条从节点u到节点v的边，则v.f < u.f。

&emsp;&emsp;考虑算法dfs(G)所探索的任意一条边(u,v)，当这条边被探索时，节点v不可能是灰色，如果那样的话，节点v将是节点u的祖先，这样(u,v)将是一条后向边，与上面的引理相矛盾，因此节点v要么是白色，要么是黑色的，如果节点v是白色的，它将成为节点u的后代，因此u.f > v.f；如果节点是黑色的，则对其全部的处理都已经完成，因此v.f已经被设置。因为还需要对节点u进行探索，u.f尚需要设定。但一旦我们对u.f进行设定，其数值一定比v.f大，既u.f > v.f，因此对于任意一条边(u,v)，有u.f > v.f。

In [3]:
# 代码实现拓扑排序
class Graph:
    def __init__(self):
        self.V = []
        
class Vertex:
    def __init__(self,x):
        self.key = x
        self.colot = 'white'
        self.d = float('Inf')
        self.f = float('Inf')
        self.prev = None
        self.adj = []
        self.next = None
        
class Solution:
    def dfs(self,G):
        for u in G.V:
            u.color = 'white'
            u.prev = None
        global time
        time = 0
        for u in G.V:
            if u.color == 'white':
                self.dfs_visit(G,u)
                
    def dfs_visit(self,G,u):
        global time
        time = time + 1
        u.d = time
        u.color = 'gray'
        for v in u.adj:
            if v.color == 'white':
                self.dfs_visit(G,v)
                v.prev = u
                
        u.color = 'black'
        time = time + 1
        u.f = time
        
    def topological_sort(self,G):
        linked_list = Vertex('#')
        self.dfs(G)
        G.V.sort(key=lambda v:v.f)
        for v in G.V:
            v.next = linked_list.next
            linked_list.next = v
        return linked_list

In [6]:
undershorts = Vertex('undershorts') 
socks = Vertex('socks') 
pants = Vertex('pants') 
shoes = Vertex('shoes') 
belt = Vertex('belt')
shirt = Vertex('shirt') 
tie = Vertex('tie')
jacket = Vertex('jacket') 
watch = Vertex('watch')

undershorts.adj = [pants, shoes]
socks.adj = [shoes]
pants.adj = [belt, shoes]
shoes.adj = []
belt.adj = [jacket]
shirt.adj = [belt, tie]
tie.adj = [jacket]
jacket.adj = []
watch.adj = []

G = Graph()
G.V = [undershorts,socks,pants,shoes,belt,shirt,tie,jacket,watch]

m = Solution()
Sort_List = m.topological_sort(G)
p = Sort_List
while p.next != None:
    print(p.next.key + '| ' + str(p.next.f))
    p = p.next

watch| 18
shirt| 16
tie| 15
socks| 12
undershorts| 10
pants| 9
shoes| 8
belt| 6
jacket| 5


#### 4.深度优先_广度优先 leetcode例题

| 基础  |                           |
|-----|---------------------------|
| 200 | Number of Islands         |
| 286 | Walls and Gates           |
| 130 | Surrounded Regions        |
| 339 | Nested List Weight Sum    |
| 364 | Nested List Weight Sum II |
| 127 | Word Ladder               |
| 51  | N\-Queens                 |
| 52  | N\-Queens II              |
| 126 | Word Ladder II            |

&emsp;&emsp;其中word_ladder这个问题比较难 ，126题特别难，至今没有思路，之后要重点复习

#### 5.拓扑排序 leetcode例题

| 基础  |                           |
|-----|---------------------------|
| 207 | Course Schedule           |
| 210 | Course Schedule II        |
| 269 | Alien Dictionary          |